In [1]:
import sys, os
import rasterio, boto3

import pandas as pd
import geopandas as gpd
import GOSTRocks.rasterMisc as rMisc

from GOSTRocks.misc import tPrint
from botocore.config import Config
from botocore import UNSIGNED
from matplotlib import pyplot

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [2]:
# Define input and output
in_adm_file = "/home/public/Data/COUNTRY/SDN/ADMIN/geoBoundaries-SDN-ADM1.geojson"

output_folder = "/home/wb411133/projects/SDN_NTL_Zonal"
out_ntl_folder = os.path.join(output_folder, "NTL_DATA")
out_ntl_zonal  = os.path.join(output_folder, "NTL_Zonal")
for f in [output_folder, out_ntl_folder, out_ntl_zonal]:
    if not os.path.exists(f):
        os.makedirs(f)

# Get list of NTL composites

In [3]:
# Define baseline variables
bucket = 'globalnightlight'
region = 'us-east-1'
s3client = boto3.client('s3', region_name=region, config=Config(signature_version=UNSIGNED))

In [4]:
# Loop through the S3 bucket and get all the keys for files that are radiance tifs and json files
more_results = True
try:
    del(token)
except:
    pass
loops = 0
good_tif = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(Bucket=bucket, ContinuationToken=token, Prefix='composites')
    else:
        objects = s3client.list_objects_v2(Bucket=bucket)
    more_results = objects['IsTruncated']
    if more_results:
        token = objects['NextContinuationToken']
    loops += 1
    for res in objects['Contents']:
        if res['Key'].endswith('avg_rade9.tif') and ("slcorr" in res['Key']):
            good_tif.append(res)        

Completed loop: 0
Completed loop: 1


In [5]:
for ntl_layer in good_tif:
    aws_path = os.path.join("https://globalnightlight.s3.amazonaws.com/", ntl_layer['Key'])
    tPrint(os.path.basename(aws_path))
    inNtl = rasterio.open(aws_path)
    inAdm = gpd.read_file(in_adm_file)

    ntl_file = os.path.join(out_ntl_folder, os.path.basename(aws_path))
    #if not os.path.exists(ntl_file):
    #    rMisc.clipRaster(inNtl, inAdm, ntl_file)

    ntl_zonal_file = os.path.join(out_ntl_zonal, os.path.basename(aws_path).replace(".tif", ".csv"))
    if not os.path.exists(ntl_zonal_file):
        res = rMisc.zonalStats(inAdm, inNtl, minVal=0)
        res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
        res['gID'] = inAdm['shapeID']
        res.to_csv(ntl_zonal_file)

10:07:17	DNB_npp_20120119-20120131_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:18	DNB_npp_20120201-20120229_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:18	DNB_npp_20120301-20120331_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:19	DNB_npp_20120401-20120430_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:19	DNB_npp_20120501-20120531_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:20	DNB_npp_20120601-20120630_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:20	DNB_npp_20120701-20120731_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:21	DNB_npp_20120801-20120831_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:21	DNB_npp_20120901-20120930_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:22	DNB_npp_20121001-20121031_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:22	DNB_npp_20121101-20121130_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:23	DNB_npp_20121201-20121231_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:23	DNB_npp_20130101-20130131_global_vcm-slcorr_v10_rp2.avg_rade9.tif
10:07:24	DNB_npp_20130201

10:08:43	DNB_npp_20210401-20210430_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:08:47	DNB_npp_20210501-20210531_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:08:51	DNB_npp_20210601-20210630_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:08:54	DNB_npp_20210701-20210731_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:08:58	DNB_npp_20210901-20210930_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:01	DNB_npp_20211001-20211031_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:05	DNB_npp_20211101-20211130_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:09	DNB_npp_20211201-20211231_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:13	DNB_npp_20220101-20220131_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:17	DNB_npp_20220201-20220228_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:20	DNB_npp_20220301-20220331_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:24	DNB_npp_20220401-20220430_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:28	DNB_npp_20220501-20220531_global_ecm-slcorr_v10_ops.avg_rade9.tif
10:09:32	DNB_npp_20220701

In [6]:
# Stack results from csv analysis
try:
    del final
except:
    pass
for zonal_file in os.listdir(out_ntl_zonal):
    date = zonal_file.split("_")[2].split("-")[0]
    inZ = pd.read_csv(os.path.join(out_ntl_zonal, zonal_file), index_col=0)
    try:
        final[date] = inZ['SUM']
    except:
        final = inZ.copy().drop(['SUM', 'MIN', 'MAX', 'MEAN'], axis=1)
        final[date] = inZ['SUM']    

In [8]:
final = final.reindex(sorted(final.columns), axis=1)

,20120119,20120201,20120301,20120401,20120501,20120601,20120701,20120801,20120901,20121001,...,20211201,20220101,20220201,20220301,20220401,20220501,20220701,20221101,20221201,gID
0,16009.630859,15708.779297,19667.929688,18604.400391,17152.019531,6803.899902,10974.670898,4960.189941,7106.949707,12428.360352,...,30076.767578,14143.029297,15565.809570,16255.959961,17075.158203,14990.400391,9536.879883,18321.929688,18085.039062,SDN-ADM1-86620642B88867350
1,31526.759766,37471.628906,50414.367188,57956.871094,48218.488281,32942.886719,31350.355469,42101.332031,47243.820312,40938.109375,...,57088.894531,53589.863281,61010.113281,65855.054688,85945.796875,70373.585938,43363.710938,66063.546875,63040.218750,SDN-ADM1-86620642B51960613
2,37275.410156,49083.660156,66366.906250,58765.363281,31393.513672,23229.630859,4668.399902,33358.378906,33836.363281,49282.000000,...,53466.023438,40502.945312,58383.843750,64418.445312,59226.785156,59180.628906,41576.804688,55317.503906,58112.632812,SDN-ADM1-86620642B91082353
3,37525.675781,34870.738281,52322.128906,53402.199219,47605.464844,24471.207031,29830.824219,44132.523438,41895.855469,37876.128906,...,34224.539062,29992.611328,37031.945312,47972.128906,47342.195312,54921.531250,19529.406250,39623.730469,39301.589844,SDN-ADM1-86620642B3923818
4,56570.867188,79500.007812,94240.031250,102989.648438,78764.015625,38462.625000,53041.648438,64104.308594,44218.691406,69580.054688,...,66852.257812,51124.089844,69868.625000,72908.695312,79257.617188,78559.335938,38304.906250,87318.578125,68509.406250,SDN-ADM1-86620642B82453041
5,50000.742188,67499.617188,95138.898438,115011.945312,84489.593750,64705.117188,45907.039062,64853.812500,88851.515625,78920.648438,...,68332.648438,56617.628906,78558.718750,91569.859375,103111.203125,105660.484375,67866.726562,80182.578125,71639.171875,SDN-ADM1-86620642B11858188
6,42065.953125,58470.753906,78954.820312,99519.148438,78589.679688,55025.398438,46792.109375,67817.843750,75381.062500,69668.718750,...,55396.152344,46697.527344,58044.804688,70156.359375,85619.445312,86511.546875,53923.839844,66178.039062,57855.933594,SDN-ADM1-86620642B90817386
7,83243.406250,84473.742188,102303.570312,105328.976562,86874.695312,79194.507812,86686.953125,81972.562500,83654.609375,86158.148438,...,142899.890625,144135.343750,152467.718750,153804.703125,185317.843750,150796.593750,115952.531250,159281.468750,166012.218750,SDN-ADM1-86620642B11096066
8,382469.625000,408802.843750,534176.625000,660060.562500,563621.062500,398685.125000,374821.562500,512014.031250,563302.750000,491259.593750,...,378384.093750,328311.875000,389497.062500,531064.562500,576847.312500,604416.312500,271005.812500,447167.593750,417729.375000,SDN-ADM1-86620642B1690804
9,189714.781250,219799.656250,313646.250000,365140.187500,298512.156250,220623.656250,208266.375000,281978.125000,334905.437500,265602.437500,...,224475.140625,172897.062500,227841.515625,279847.250000,323551.187500,326019.687500,199601.062500,274902.500000,234409.046875,SDN-ADM1-86620642B69992162


In [9]:
final.to_csv(os.path.join(output_folder, 'NTL_VIIRS_ZONAL.csv'))